In [100]:
import pandas as pd
from os.path import exists
import time
import datetime
from pathlib import Path
import numpy as np

# read files from directory
file_list = []

for file in sorted(os.listdir('static/img/portfolio')):
    file_list.append(file)

df_dir = pd.DataFrame(file_list, columns=['Files'])
df_old = pd.read_csv('processed.csv')

# get list of files not processed yet
df_new = df_dir.merge(df_old, how='left', indicator=True)
df_new = df_new[df_new['_merge']=='left_only'].drop(columns='_merge')

# process files
for file in df_new['Files']:
    
    file_path = str(Path('content/portfolio/' + file).with_suffix('.md'))
    img_path = str(Path('img/portfolio/' + file).with_suffix('.png'))
    
    last_time = open("lastposttime.txt").read()
    new_time = datetime.datetime.strptime(last_time, '%Y-%m-%d %H:%M:%S') + datetime.timedelta(seconds=3600)
    
    content = '+++\ndate = "' + str(new_time.strftime("%Y-%m-%dT%H:%M:%S+05:00")) + '"\ndraft = false\nimage = "' + img_path + '"\nshowonlyimage = true\ntitle = ""\n+++\n\n<!--more-->\n![Post image]('+ img_path +')'
    with open(file_path, 'w') as f:
        f.write(content)

    with open("lastposttime.txt", 'w') as f:
        f.write(str(new_time))

    with open('processed.csv','a') as f:
        f.write('\n'+img_path)